# Construct of general ANM matrix using PP
using PP for ANM is quite a lot more complicated and tedious than all-electron systems.
This note book attempt to create a readable/reusable script to construct PP ANM matrix

In [1]:
import qctoolkit as qtk
import numpy as np

/home/samio/local/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# reuse code from PRM paper
cyl_map = [
        ['si', 5.431, [14, 14]],
        ['ge', 5.658, [32, 32]],
        ['sige', 5.432, [14, 32]],
        ['sn', 6.4892, [50, 50]],
        ['snsi', 5.961, [14, 50]],
        ['gesn', 6.0758, [32, 50]],
        ['gaas', 5.6535, [31, 33]],
        ['alp', 5.4635, [13, 15]],
        ['alas', 5.660, [13, 33]],
        ['alsb', 6.1355, [13, 51]],
        ['gap', 5.451, [31, 15]],
        ['gasb', 6.09, [31, 51]],
        ['inp', 5.86, [49, 15]],
        ['inas', 6.05, [49, 33]],
        ['insb', 6.47, [49, 51]],

]
print len(cyl_map)

15


In [3]:
# only CPMD PP interface is ready at the moment
# only Gamma-point calculations are required
qmsetting = {
    'program': 'cpmd',
    'kmesh': [1,1,1],
    'link_dep': True,
}

In [4]:
mol_base = qtk.Molecule('xyz/ge.xyz')

In [5]:
# reference calculations with lattice constant scan
mols = []
count = 0
for info_mol in cyl_map:
    count += 1
    for info_a in cyl_map:
        new = mol_base.copy()
        new.name = info_mol[0] + '_a-' + info_a[0]
        new.setCelldm([info_a[1], info_a[1], info_a[1], 0, 0, 0])
        for i in range(new.N):
            if new.Z[i] != info_mol[2][i]:
                new.setAtoms(i, Z=info_mol[2][i])
        mols.append(new)
print count
inps_ref = [qtk.QMInp(mol, **qmsetting) for mol in mols]
print len(inps_ref)
print inps_ref[20]

15
225
ge_a-gesn: cpmd


# ANM space and finite difference matrix
ANM space is spanned by the PP parameters. For FCC primitive cell of two atoms, there are 30 parameters (15 per atoms).

The goal is the construct matrix elements 
$$
\begin{array}{rcl}
\mathbf{H}_{ij}&=&\frac{\partial^2 E}{\partial\sigma_i\partial\sigma_j}\\
&=&\frac{\partial}{\partial\sigma_i}\Big(\frac{\partial E}{\partial\sigma_j}\Big)\\
&\approx&
\frac{\partial}{\partial\sigma_i}\Big(\frac{E(\sigma_i, \sigma_j+\Delta\sigma_j) - E(\sigma_i, \sigma_j)}{\Delta\sigma_j}\Big)\\
&\approx&
\frac{1}{\Delta\sigma_i}\Bigg(\frac{E(\sigma_i+\Delta\sigma_i, \sigma_j+\Delta\sigma_j) - E(\sigma_i+\Delta\sigma_i, \sigma_j)}{\Delta\sigma_j}
- \frac{E(\sigma_i, \sigma_j+\Delta\sigma_j) - E(\sigma_i, \sigma_j)}{\Delta\sigma_j}\Bigg).
\end{array}
$$

That is, there are four finite difference calculations required for each of the matrix elements:
$E(\sigma_i+\Delta\sigma_i, \sigma_j+\Delta\sigma_j)$, 
$E(\sigma_i+\Delta\sigma_i, \sigma_j)$, 
$E(\sigma_i, \sigma_j+\Delta\sigma_j)$, 
$E(\sigma_i, \sigma_j)$, 
where only the first term is unique for each element.

Note that the finite difference formula is different for diagonal terms $\mathbf{H}_{ii} = \frac{E(\sigma_i+\Delta\sigma_i) - 2E(\sigma_i) + E(\sigma_i-\Delta\sigma_i)}{\Delta\sigma_i^2}$.

The required finite difference calculations are
* $E(\sigma_i+\Delta\sigma_i, \sigma_j+\Delta\sigma_j)$: $N^2$ calculations
* $E(\sigma_i+\Delta\sigma_i)$: $N$ calculations
* $E(\sigma_i-\Delta\sigma_i)$: $N$ calculations
* $E(\sigma_i, \sigma_j)$: 1 calculation

which added up to $N^2+2N+1$ calculations, where $N$ is the number of parameters in the system.

# Function interface
The following functions are required for reusable/extensive code

## _crystal to parameter space map_: cyl2par
input: crystal object

output: parameter space coordinate

## _parameter spsce to crystal map_: par2cyl
input: parameter space coordinate

output: crystal object

## _finite difference crystal construction_: FDcyl
input: finite difference indices, step size

output: QMInp calculation object

__NOTE__: for the construction of FD matrix, the index information will be written on the file name for lazy processing

## _finite difference matrix construction_: H_FD
input: finite difference calculation folder path __with fomated index file name__

output: finite difference matrix as 2D numpy array

### TODO:
implement CPMD interface to take PP object as input and write PP file on the fly __DONE__

In [7]:
def cyl2par(mol, size=[1, 3]):
    par = []
    for a in mol.type_list:
        crd = qtk.PP(a, size=size)
        vec = crd.vectorize()
        par.append([vec[0][1:], a, vec[0][0]])
    return par

def par2cyl(par, mol_base=mol_base, size=[1, 3]):
    mol = mol_base.copy()
    mol.sort(order='xyz')
    for i, crd in enumerate(par):
        print crd
        pp = qtk.PP(crd[1], size=size)
        vec = [crd[2]]
        vec.extend(crd[0])
        pp.unvectorize(vec, size[0], range(size[1], 0, -1))
        pp.name = crd[1] + '%02d' % i
        mol.setAtoms(i, string=pp)
    return mol

mol = par2cyl(cyl2par(mol_base))
inp = qtk.QMInp(mol, **qmsetting)
inp.write()

[array([ 0.54      ,  0.        ,  0.42186518,  7.51024121, -0.58810836,
       -1.4479758 , -1.59588819,  3.73865744, -2.96746735,  0.56752887,
        0.91385969,  0.54687534, -0.64707163,  0.81391394,  0.19717731]), 'Ge', 4.0]
[array([ 0.54      ,  0.        ,  0.42186518,  7.51024121, -0.58810836,
       -1.4479758 , -1.59588819,  3.73865744, -2.96746735,  0.56752887,
        0.91385969,  0.54687534, -0.64707163,  0.81391394,  0.19717731]), 'Ge', 4.0]
&INFO
 ge
&END

&CPMD
 MIRROR
 BENCHMARK
  1 0 0 0 0 0 0 0 0 0
 OPTIMIZE WAVEFUNCTION
 CONVERGENCE ORBITAL
  1.00E-05
 MAXITER
  1000
 CENTER MOLECULE OFF
 MEMORY BIG
&END

&DFT
 FUNCTIONAL PBE
&END

&SYSTEM
 CELL VECTORS
     0.000000    2.829000    2.829000
     2.829000    0.000000    2.829000
     2.829000    2.829000    0.000000
 ANGSTROM
 CUTOFF
  100.0
 KPOINTS MONKHORST-PACK
  1 1 1
&END

&ATOMS
*Ge01.psp
 LMAX=F
   1
     1.4145   1.4145   1.4145
*Ge00.psp
 LMAX=F
   1
     0.0000   0.0000   0.0000
&END


In [26]:
def FDcyl(mol, i, j, step, size=[1,3]):
    """
    input:  molecule object: where the FD perturbation will be introduced
            i, j: integer indices for the perturbation matrix row and column number
            step: absolute step size for finite difference perturbation
            size: PP configuration space
    output: molecule object with appropreate PP and file names attached
    """
    mol = mol.copy()
    inds = [i, j]
    ind_font = ['I', 'J']
    for t, s in enumerate([i, j]):
        I, ip = divmod(s, 15)
        inds.append(I)
        inds.append(ip)
        if type(mol.string[I]) is type(qtk.PP()):
            pp = mol.string[I]
        else:
            pp = qtk.PP(mol.type_list[I], size=size)
            pp.name = mol.type_list[I]
        vec = pp.vectorize()[0]
        new_pp = pp.copy()
        vec[1+ip] += step
        new_pp.unvectorize(vec, size[0], range(size[1], 0, -1))
        new_pp.name += '_%s%02dp%02d' % (ind_font[t], I, ip)
        mol.setAtoms(I, string=new_pp)
    mol.name += "_i%02dj%02d-I%02dp%02d-J%02dp%02d" % tuple(inds)
    return mol

mol_test = FDcyl(mol_base, 0, 1, 100)
inp = qtk.QMInp(mol_test, **qmsetting)
inp.write('PPTest', overwrite=True)

QMInp: Ge_q4_pbe.psp is linked



## Test of PP interface

In [25]:
pp = qtk.PP('Ge', size=[1,3])
print pp
print pp.param
v = pp.vectorize()[0]
for i in range(2, 7):
    v[i] = 10
new = pp.copy().unvectorize(v, 1, [3,2,1])

print pp.param
print new.param

Cn=1, l_max=3, parameters=15

{'l_max': 3, 'Cn': 1, 'xc': 'pbe', 'Ci': [0], 'r_nl': [0.421865178, 0.567528874, 0.813913943], 'r_loc': 0.54, 'ZV': 4.0, 'Z': 32.0, 'h_ij': [array([[ 7.51024121, -0.58810836, -1.4479758 ],
       [-0.58810836, -1.59588819,  3.73865744],
       [-1.4479758 ,  3.73865744, -2.96746735]]), array([[ 0.91385969,  0.54687534],
       [ 0.54687534, -0.64707163]]), array([[0.19717731]])]}
{'l_max': 3, 'Cn': 1, 'xc': 'pbe', 'Ci': [0], 'r_nl': [0.421865178, 0.567528874, 0.813913943], 'r_loc': 0.54, 'ZV': 4.0, 'Z': 32.0, 'h_ij': [array([[ 7.51024121, -0.58810836, -1.4479758 ],
       [-0.58810836, -1.59588819,  3.73865744],
       [-1.4479758 ,  3.73865744, -2.96746735]]), array([[ 0.91385969,  0.54687534],
       [ 0.54687534, -0.64707163]]), array([[0.19717731]])]}
{'l_max': 3, 'Cn': 1, 'xc': 'pbe', 'Ci': [10.0], 'r_nl': [10.0, 0.567528874, 0.813913943], 'r_loc': 0.54, 'ZV': 4.0, 'Z': 32.0, 'h_ij': [array([[10.        , 10.        , 10.        ],
       [10.        

In [8]:
v.shape

(16,)

In [14]:
mol = mols[3].copy()
mol.extend([2,2,2])
mol.sort(order='xyz').R

array([[ 0.    ,  0.    ,  0.    ],
       [ 0.    ,  0.    , 12.9784],
       [ 0.    , 12.9784,  0.    ],
       [ 0.    , 12.9784, 12.9784],
       [ 3.2446,  3.2446,  3.2446],
       [ 3.2446,  3.2446, 16.223 ],
       [ 3.2446, 16.223 ,  3.2446],
       [ 3.2446, 16.223 , 16.223 ],
       [12.9784,  0.    ,  0.    ],
       [12.9784,  0.    , 12.9784],
       [12.9784, 12.9784,  0.    ],
       [12.9784, 12.9784, 12.9784],
       [16.223 ,  3.2446,  3.2446],
       [16.223 ,  3.2446, 16.223 ],
       [16.223 , 16.223 ,  3.2446],
       [16.223 , 16.223 , 16.223 ]])

In [7]:
mol.setAtoms(0, string='test')

In [8]:
mol.Z

array([ 0., 14.])

In [9]:
inp = qtk.QMInp(mol, **qmsetting)

In [19]:
inp.write()

&INFO
 si_a-sn
&END

&CPMD
 MIRROR
 BENCHMARK
  1 0 0 0 0 0 0 0 0 0
 OPTIMIZE WAVEFUNCTION
 CONVERGENCE ORBITAL
  1.00E-05
 MAXITER
  1000
 CENTER MOLECULE OFF
 MEMORY BIG
&END

&DFT
 FUNCTIONAL PBE
&END

&SYSTEM
 CELL VECTORS
     0.000000    3.244600    3.244600
     3.244600    0.000000    3.244600
     3.244600    3.244600    0.000000
 ANGSTROM
 CUTOFF
  100.0
 KPOINTS MONKHORST-PACK
  1 1 1
&END

&ATOMS
*test.psp
 LMAX=F
   2
     0.0000   0.0000   0.0000
     1.6223   1.6223   1.6223
&END


In [10]:
mols

[si_a-si,
 si_a-ge,
 si_a-sige,
 si_a-sn,
 si_a-snsi,
 si_a-gesn,
 si_a-gaas,
 si_a-alp,
 si_a-alas,
 si_a-alsb,
 si_a-gap,
 si_a-gasb,
 si_a-inp,
 si_a-inas,
 si_a-insb,
 ge_a-si,
 ge_a-ge,
 ge_a-sige,
 ge_a-sn,
 ge_a-snsi,
 ge_a-gesn,
 ge_a-gaas,
 ge_a-alp,
 ge_a-alas,
 ge_a-alsb,
 ge_a-gap,
 ge_a-gasb,
 ge_a-inp,
 ge_a-inas,
 ge_a-insb,
 sige_a-si,
 sige_a-ge,
 sige_a-sige,
 sige_a-sn,
 sige_a-snsi,
 sige_a-gesn,
 sige_a-gaas,
 sige_a-alp,
 sige_a-alas,
 sige_a-alsb,
 sige_a-gap,
 sige_a-gasb,
 sige_a-inp,
 sige_a-inas,
 sige_a-insb,
 sn_a-si,
 sn_a-ge,
 sn_a-sige,
 sn_a-sn,
 sn_a-snsi,
 sn_a-gesn,
 sn_a-gaas,
 sn_a-alp,
 sn_a-alas,
 sn_a-alsb,
 sn_a-gap,
 sn_a-gasb,
 sn_a-inp,
 sn_a-inas,
 sn_a-insb,
 snsi_a-si,
 snsi_a-ge,
 snsi_a-sige,
 snsi_a-sn,
 snsi_a-snsi,
 snsi_a-gesn,
 snsi_a-gaas,
 snsi_a-alp,
 snsi_a-alas,
 snsi_a-alsb,
 snsi_a-gap,
 snsi_a-gasb,
 snsi_a-inp,
 snsi_a-inas,
 snsi_a-insb,
 gesn_a-si,
 gesn_a-ge,
 gesn_a-sige,
 gesn_a-sn,
 gesn_a-snsi,
 gesn_a-gesn,
 gesn_a